In [111]:
import blosc
import os
import shutil
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

#### Creating data for long DFs
rating_10K_brand_data = {
          "respondent": [i for i in xrange(10000)],
          "country":["UK", "UK", "UK", "USA", "USA"] * 2000,
          "ps4_rating": [3.0, 2.0, 9.0, 7.0, 2.0] * 2000,
          "xbox_rating": [2.0, 4.0, 2.0, 8.0, 6.0] * 2000,
          "switch_rating": [3.0, 6.0, 7.0, 4.0, 5.0] * 2000
      }

rating_100K_brand_data = {
          "respondent"   : [i for i in xrange(100000)],
          "country"      : ["UK", "UK", "UK", "USA", "USA"] * 20000,
          "ps4_rating"  : [3.0, 2.0, 9.0, 7.0, 2.0] * 20000,
          "xbox_rating" : [2.0, 4.0, 2.0, 8.0, 6.0] * 20000,
          "switch_rating"  : [3.0, 6.0, 7.0, 4.0, 5.0] * 20000
      }

rating_1M_brand_data = {
          "respondent"   : [i for i in xrange(1000000)],
          "country"      : ["UK", "UK", "UK", "USA", "USA"] * 200000,
          "ps4_rating"  : [3.0, 2.0, 9.0, 7.0, 2.0] * 200000,
          "xbox_rating" : [2.0, 4.0, 2.0, 8.0, 6.0] * 200000,
          "switch_rating"  : [3.0, 6.0, 7.0, 4.0, 5.0] * 200000
      }

#### Creating data for wide DFs
rating_100_col_brand_data = {
    'col_' + str(i): [3.0, 2.0, 9.0, 7.0, 2.0] * 30 
    for i in xrange(100)
}
rating_100_col_brand_data['respondent'] = [i for i in xrange(150)]

rating_1K_col_brand_data = {
    'col_' + str(i): [3.0, 2.0, 9.0, 7.0, 2.0] * 30 
    for i in xrange(1000)
}
rating_1K_col_brand_data['respondent'] = [i for i in xrange(150)]

rating_10K_col_brand_data = {
    'col_' + str(i): [3.0, 2.0, 9.0, 7.0, 2.0] * 30 
    for i in xrange(10000)
}
rating_10K_col_brand_data['respondent'] = [i for i in xrange(150)]

#### Creating long DFs
rating_10K = pd.DataFrame(rating_10K_brand_data)
rating_10K = rating_10K.set_index(['respondent', 'country'])

rating_100K = pd.DataFrame(rating_100K_brand_data)
rating_100K = rating_100K.set_index(['respondent', 'country'])

rating_1M = pd.DataFrame(rating_1M_brand_data)
rating_1M = rating_1M.set_index(['respondent', 'country'])

###### Creating wide DFs
rating_100_col = pd.DataFrame(rating_100_col_brand_data)
rating_100_col = rating_100_col.set_index(['respondent'])

rating_1K_col = pd.DataFrame(rating_1K_col_brand_data)
rating_1K_col = rating_1K_col.set_index(['respondent'])

rating_10K_col = pd.DataFrame(rating_10K_col_brand_data)
rating_10K_col = rating_10K_col.set_index(['respondent'])


##### Setting up dirs
try:
    os.makedirs('example_parquet_files')
    os.makedirs('example_blosc_files')
except Exception as exc:
    raise exc


#### Compression function (for demonstration purposes)
def compress_df(df, data_dir):
    try:
        data_path = os.path.join('example_blosc_files', data_dir)
        os.makedirs(data_path)
        
        for col in df.columns.tolist():
            packed_array = blosc.pack_array(df[col].values)
            with open(os.path.join(data_path, "{}.bin".format(col)), 'wb') as data_file:
                data_file.write(packed_array)
    except Exception as exc:
        raise exc
    


### Snappy Compression

#### Long DFs 

In [39]:
arr_table_10K = pa.Table.from_pandas(rating_10K)

In [42]:
pq.write_table(arr_table_10K, 'example_parquet_files/rating_10K.parquet')

In [43]:
%timeit df_out = pq.read_table('example_parquet_files/rating_10K.parquet', columns=None).to_pandas()

100 loops, best of 3: 4.35 ms per loop


In [44]:
arr_table_100K = pa.Table.from_pandas(rating_100K)

In [45]:
pq.write_table(arr_table_100K, 'example_parquet_files/rating_100K.parquet')

In [7]:
%timeit df_out = pq.read_table('example_parquet_files/rating_100K.parquet', columns=None).to_pandas()

10 loops, best of 3: 32.5 ms per loop


In [46]:
arr_table_1M = pa.Table.from_pandas(rating_1M)

In [47]:
pq.write_table(arr_table_1M, 'example_parquet_files/rating_1M.parquet')

In [10]:
%timeit df_out = pq.read_table('example_parquet_files/rating_1M.parquet', columns=None).to_pandas()

1 loop, best of 3: 422 ms per loop


#### Wide DFs

In [48]:
arr_table_100_col = pa.Table.from_pandas(rating_100_col)

In [49]:
pq.write_table(arr_table_100_col, 'example_parquet_files/rating_100_col.parquet')

In [13]:
%timeit df_out = pq.read_table('example_parquet_files/rating_100_col.parquet', columns=None).to_pandas()

100 loops, best of 3: 4.21 ms per loop


In [50]:
arr_table_1K_col = pa.Table.from_pandas(rating_1K_col)

In [51]:
pq.write_table(arr_table_1K_col, 'example_parquet_files/rating_1K_col.parquet')

In [16]:
%timeit df_out = pq.read_table('example_parquet_files/rating_1K_col.parquet', columns=None).to_pandas()

10 loops, best of 3: 34.4 ms per loop


In [52]:
arr_table_10K_col = pa.Table.from_pandas(rating_10K_col)

In [53]:
pq.write_table(arr_table_10K_col, 'example_parquet_files/rating_10K_col.parquet')

In [19]:
%timeit df_out = pq.read_table('example_parquet_files/rating_10K_col.parquet', columns=None).to_pandas()

1 loop, best of 3: 366 ms per loop


### Brotli Compression

#### Long DFs

In [68]:
arr_table_10K = pa.Table.from_pandas(rating_10K)

In [69]:
pq.write_table(arr_table_10K, 'example_parquet_files/rating_10K.parquet', compression="BROTLI")

In [31]:
%timeit df_out = pq.read_table('example_parquet_files/rating_10K.parquet', columns=None).to_pandas()

100 loops, best of 3: 4.27 ms per loop


In [56]:
arr_table_100K = pa.Table.from_pandas(rating_100K)

In [57]:
pq.write_table(arr_table_100K, 'example_parquet_files/rating_100K.parquet', compression="BROTLI")

In [34]:
%timeit df_out = pq.read_table('example_parquet_files/rating_100K.parquet', columns=None).to_pandas()

10 loops, best of 3: 28.6 ms per loop


In [58]:
arr_table_1M = pa.Table.from_pandas(rating_1M)

In [59]:
pq.write_table(arr_table_1M, 'example_parquet_files/rating_1M.parquet', compression="BROTLI")

In [37]:
%timeit df_out = pq.read_table('example_parquet_files/rating_1M.parquet', columns=None).to_pandas()

1 loop, best of 3: 421 ms per loop


#### Wide DFs

In [60]:
arr_table_100_col = pa.Table.from_pandas(rating_100_col)

In [61]:
pq.write_table(arr_table_100_col, 'example_parquet_files/rating_100_col.parquet', compression="BROTLI")

In [22]:
%timeit df_out = pq.read_table('example_parquet_files/rating_100_col.parquet', columns=None).to_pandas()

100 loops, best of 3: 4.54 ms per loop


In [64]:
arr_table_1K_col = pa.Table.from_pandas(rating_1K_col)

In [65]:
pq.write_table(arr_table_1K_col, 'example_parquet_files/rating_1K_col.parquet', compression="BROTLI")

In [25]:
%timeit df_out = pq.read_table('example_parquet_files/rating_1K_col.parquet', columns=None).to_pandas()

10 loops, best of 3: 37.6 ms per loop


In [66]:
arr_table_10K_col = pa.Table.from_pandas(rating_10K_col)

In [67]:
pq.write_table(arr_table_10K_col, 'example_parquet_files/rating_10K_col.parquet', compression="BROTLI")

In [28]:
%timeit df_out = pq.read_table('example_parquet_files/rating_10K_col.parquet', columns=None).to_pandas()

1 loop, best of 3: 406 ms per loop


### Blosc Compressed Columns

#### Long DFs

In [112]:
compress_df(rating_10K, 'rating_10K')

In [113]:
compress_df(rating_100K, 'rating_100K')

In [114]:
compress_df(rating_1M, 'rating_1M')

#### Wide DFs

In [115]:
compress_df(rating_100_col, 'rating_100_col')

In [116]:
compress_df(rating_1K_col, 'rating_1K_col')

In [117]:
compress_df(rating_10K_col, 'rating_10K_col')

In [118]:
shutil.rmtree('example_parquet_files')
shutil.rmtree('example_blosc_files')